In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [24]:
import tensorflow as tf
sess = tf.InteractiveSession()

#Placeholde definition
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
#Variable definition
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
sess.run(tf.global_variables_initializer())

#implementing softmax regression model
y = tf.matmul(x,W) + b
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

#train the model
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
for _ in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_:batch[1]})

c:\users\varun_bawa\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\client\session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [25]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_:mnist.test.labels}))

0.9169


In [26]:
#Build a Multilayer Convolutional Network
#To create this model, we're going to need to create a lot of weights and biases.
#One should generally initialize weights with a small amount of noise for symmetry breaking, and to prevent 0 gradients.

In [27]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#Convolution and Pooling(Using TF vanilla)
#Some Info
#TensorFlow also gives us a lot of flexibility in convolution and pooling operations.
#How do we handle the boundaries? What is our stride size? In this example, we're always going to choose the vanilla version.
#Our convolutions uses a stride of one and are zero padded so that the output is the same size as the input.
#Our pooling is plain old max pooling over 2x2 blocks.
#To keep our code cleaner, let's also abstract those operations into functions.

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [28]:
#Our First Convolutional Layer
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

#reshape x to a 4d tensor, with the second and third dimensions corresponding to image width and height, 
#and the final dimension corresponding to the number of color channels.
x_image = tf.reshape(x, [-1, 28, 28, 1])

#We then convolve x_image with the weight tensor, add the bias, apply the ReLU function, and finally max pool.
#The max_pool_2x2 method will reduce the image size to 14x14.
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_conv1 = max_pool_2x2(h_conv1)

In [32]:
#Our Second Convolutional Layer

#In order to build a deep network, we stack several layers of this type.
#The second layer will have 64 features for each 5x5 patch.
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [33]:
#Densely Connected Layer
#Now that the image size has been reduced to 7x7, we add a fully-connected layer with 1024 neurons 
#to allow processing on the entire image. We reshape the tensor from the pooling layer into a batch of vectors, 
#multiply by a weight matrix, add a bias, and apply a ReLU

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [34]:
#Dropout
#To reduce overfitting, we will apply dropout before the readout layer. 
#We create a placeholder for the probability that a neuron's output is kept during dropout.
#This allows us to turn dropout on during training, and turn it off during testing.
#TensorFlow's tf.nn.dropout op automatically handles scaling neuron outputs in addition to masking them, 
#so dropout just works without any additional scaling.

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [35]:
#Readout Layer
#Finally, we add a layer, just like for the one layer softmax regression above.

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [37]:
#Training and Evaluate the model
#How well does this model do? To train and evaluate it we will use code that is nearly identical 
#to that for the simple one layer SoftMax network above.

#The differences are that:

#We will replace the steepest gradient descent optimizer with the more sophisticated ADAM optimizer.
#We will include the additional parameter keep_prob in feed_dict to control the dropout rate.
#We will add logging to every 100th iteration in the training process.

#Feel free to go ahead and run this code, but it does 20,000 training iterations and may take a while 
#(possibly up to half an hour), depending on your processor.

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())

for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
        print("Steps: %s, training accuracy: %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    
print("Final Test Accuracy: %g"%accuracy.eval(feed_dict={x: mnist.test.images, y_:mnist.test.labels, keep_prob: 1.0}))

Steps: 0, training accuracy: 0.04
Steps: 100, training accuracy: 0.78
Steps: 200, training accuracy: 0.92
Steps: 300, training accuracy: 0.96
Steps: 400, training accuracy: 0.88
Steps: 500, training accuracy: 0.96
Steps: 600, training accuracy: 0.94
Steps: 700, training accuracy: 0.94
Steps: 800, training accuracy: 0.92
Steps: 900, training accuracy: 0.98
Steps: 1000, training accuracy: 0.98
Steps: 1100, training accuracy: 0.98
Steps: 1200, training accuracy: 0.98
Steps: 1300, training accuracy: 1
Steps: 1400, training accuracy: 0.98
Steps: 1500, training accuracy: 0.96
Steps: 1600, training accuracy: 0.94
Steps: 1700, training accuracy: 0.96
Steps: 1800, training accuracy: 0.98
Steps: 1900, training accuracy: 1
Steps: 2000, training accuracy: 1
Steps: 2100, training accuracy: 1
Steps: 2200, training accuracy: 0.98
Steps: 2300, training accuracy: 1
Steps: 2400, training accuracy: 1
Steps: 2500, training accuracy: 0.96
Steps: 2600, training accuracy: 0.96
Steps: 2700, training accuracy: